In [0]:
#Librerías

import requests
import os
import time
import pandas as pd
from io import StringIO
from datetime import datetime, date, timedelta
import numpy as np
import logging

In [0]:
# Logger para avisar

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s', datefmt='%H:%M:%S', force=True)
logger = logging.getLogger("ETL_VOLUMES")

# Paths y carpetas para Nasa 

BASE_VOLUME_PATH_NASA = "/Volumes/fire_risk_project/00_landing/nasa_files"
PATH_NASA = os.path.join(BASE_VOLUME_PATH_NASA, "nasa_firms")

# Directorios

try:
    os.makedirs(PATH_NASA, exist_ok=True)
    logger.info("Configurado")
except Exception as e:
    logger.warning(f"Aviso: {e}")

In [0]:
# Credenciales para la extracción

# Credenciales NASA
NASA_API_KEY = "f3a52021332f5e6530c52aaf0dffe569" 
MAP_KEY = NASA_API_KEY 
SOURCE = "VIIRS_SNPP_SP"
AREA = "-65.0,-32.5,-64.2,-30.5"

# Box de extracción: Sierras de Córdoba

In [0]:
# Función extracción NASA FIRMS

def etl_nasa(start_date, end_date):
    logger.info(f"inicio: ({start_date} -> {end_date})")
    current_date = start_date
    days_per_req = 5 # Es el máximo que me deja extraer la API
    
    while current_date <= end_date:
        file_name = f"nasa_{current_date.strftime('%Y%m%d')}.csv"
        final_path = os.path.join(PATH_NASA, file_name)
        
        if os.path.exists(final_path):
            logger.info(f"{file_name} Salteado.")
            current_date += timedelta(days=days_per_req)
            continue

        url = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{MAP_KEY}/{SOURCE}/{AREA}/{days_per_req}/{current_date}"
        
        try:
            response = requests.get(url)
            if response.status_code == 200 and "Invalid" not in response.text:
                with open(final_path, "w") as f:
                    f.write(response.text)
                logger.info(f"{file_name} Guardado")
            else:
                logger.warning(f"{current_date} No se ejecutó.")
        except Exception as e:
            logger.error(f"Error en {current_date}: {e}")
        
        current_date += timedelta(days=days_per_req)
        time.sleep(1)

logger.info("Función NASA preparada")

In [0]:
# Ejecuciones para período 2020 - 2025

inicio_nasa = date(2020, 1, 1)
fin_nasa = date(2025, 12, 30)

etl_nasa(inicio_nasa, fin_nasa)

In [0]:
# Verificación

try:
    archivos_nasa = os.listdir(PATH_NASA)
    archivos_nasa.sort()
    count_nasa = len(archivos_nasa)
    print(f"Archivos: {count_nasa}")
    if count_nasa > 0:
        print(f"Primer archivo: {archivos_nasa[0]}")
        print(f"Último archivo: {archivos_nasa[-1]}") # Verificamos tamaño del último para ver que no esté vacío
        print(f"Tamaño último archivo: {os.path.getsize(os.path.join(PATH_NASA, archivos_nasa[-1]))} bytes")
    else:
        print("Error en la extracción, no hay archivos de NASA FIRMS.")
except Exception as e:
    print(f"Error en la verificación NASA: {e}")

In [0]:
# Finalizado proceso de extracción de NASA FIRMS para el período 2020-2025